In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/FYP_Python_Notebooks')
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Mounted at /content/drive
Sun Feb 18 05:50:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------

In [ ]:
pwd

'/content/drive/MyDrive/FYP_Python_Notebooks'

In [ ]:
pip install tsfresh

In [ ]:
import pandas as pd
import os
import numpy as np
import sys
import tsfresh as tsf
from tsfresh.utilities.dataframe_functions import impute

In [ ]:
df_outf = pd.read_csv('combined_demand_df_1h.csv')
df_outf = df_outf.drop('time_slot', axis=1)

df_inf = pd.read_csv('combined_supply_df_1h.csv')
df_inf = df_inf.drop('time_slot', axis=1)

if(df_outf.shape[0] != df_inf.shape[0]): # check if dem and inf are same length
    print("outflow and inflow are NOT of same length")
    sys.exit()

In [ ]:
df_outf

,4,12,13,24,41,42,43,45,48,50,...,237,238,239,243,244,246,249,261,262,263
0,3.0,0.0,3.0,7.0,5.0,2.0,22.0,4.0,81.0,7.0,...,35.0,39.0,28.0,0.0,2.0,9.0,20.0,2.0,15.0,54.0
1,1.0,0.0,2.0,4.0,22.0,14.0,6.0,0.0,72.0,20.0,...,16.0,27.0,44.0,1.0,3.0,12.0,32.0,6.0,18.0,49.0
2,2.0,0.0,2.0,2.0,11.0,9.0,5.0,0.0,63.0,14.0,...,17.0,23.0,21.0,0.0,3.0,13.0,17.0,5.0,7.0,27.0
3,3.0,0.0,0.0,2.0,3.0,7.0,2.0,1.0,31.0,5.0,...,7.0,10.0,8.0,0.0,1.0,3.0,10.0,1.0,2.0,12.0
4,0.0,0.0,0.0,2.0,8.0,4.0,0.0,0.0,20.0,8.0,...,2.0,4.0,6.0,0.0,2.0,4.0,3.0,2.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24808,2.0,0.0,37.0,6.0,27.0,4.0,51.0,12.0,140.0,26.0,...,339.0,179.0,199.0,1.0,0.0,105.0,147.0,23.0,80.0,126.0
24809,5.0,0.0,37.0,10.0,12.0,1.0,88.0,7.0,128.0,19.0,...,394.0,119.0,191.0,1.0,1.0,93.0,159.0,26.0,67.0,112.0
24810,3.0,0.0,27.0,10.0,4.0,4.0,49.0,7.0,161.0,26.0,...,281.0,76.0,176.0,0.0,0.0,88.0,184.0,16.0,35.0,99.0
24811,9.0,0.0,14.0,9.0,12.0,4.0,37.0,11.0,175.0,24.0,...,183.0,36.0,94.0,0.0,0.0,53.0,190.0,12.0,21.0,60.0


In [ ]:
zone_significant_lags = {'4': [1, 167, 142, 22], '12': [1, 21, 24, 4], '13': [1, 23, 17, 5], '24': [1, 23, 168, 143], '41': [1, 23, 168, 144], '42': [1, 23, 168, 144], '43': [1, 4, 25, 21], '45': [1, 23, 167, 19], '48': [1, 25, 22, 18], '50': [1, 23, 19, 167], '68': [1, 19, 23, 4], '74': [1, 23, 168, 20], '75': [1, 5, 23, 14], '79': [1, 142, 166, 25], '87': [1, 18, 23, 5], '88': [1, 5, 23, 18], '90': [1, 19, 25, 22], '100': [1, 23, 167, 143], '107': [1, 25, 19, 22], '113': [1, 18, 4, 23], '114': [1, 167, 22, 142], '116': [1, 23, 168, 144], '120': [1, 82, 78, 98], '125': [1, 23, 19, 143], '127': [1, 24, 168, 144], '128': [24, 2, 144, 72], '137': [1, 23, 5, 26], '140': [1, 25, 22, 8], '141': [1, 23, 19, 5], '142': [1, 23, 19, 5], '143': [1, 23, 5, 167], '144': [1, 23, 20, 167], '148': [1, 166, 141, 21], '151': [1, 23, 5, 168], '152': [1, 23, 168, 144], '153': [168, 23, 2, 26], '158': [1, 167, 22, 142], '161': [1, 25, 4, 169], '162': [1, 25, 22, 17], '163': [1, 5, 25, 19], '164': [1, 18, 5, 23], '166': [1, 23, 5, 20], '170': [1, 25, 19, 22], '186': [1, 23, 20, 168], '194': [1, 24, 5, 21], '202': [1, 23, 168, 26], '209': [1, 23, 20, 167], '211': [1, 23, 167, 143], '224': [1, 24, 21, 168], '229': [1, 23, 18, 11], '230': [1, 23, 18, 169], '231': [1, 23, 19, 5], '232': [1, 22, 167, 142], '233': [1, 23, 19, 5], '234': [1, 4, 18, 25], '236': [1, 25, 4, 13], '237': [1, 25, 4, 17], '238': [1, 23, 4, 20], '239': [1, 4, 23, 19], '243': [1, 24, 21, 168], '244': [1, 24, 16, 168], '246': [1, 25, 22, 19], '249': [1, 25, 21, 142], '261': [1, 4, 20, 23], '262': [1, 23, 20, 167], '263': [1, 23, 19, 167]}

In [ ]:
class create_raw_features(object):
    #def __init__(self,):

    def create_lag_features(ts,lags_list):
        lag_ts0 = ts.shift(periods = lags_list[0])
        lag_ts1 = ts.shift(periods = lags_list[1])
        lag_ts2 = ts.shift(periods = lags_list[2])
        lag_ts3 = ts.shift(periods = lags_list[3])
        return lag_ts0,lag_ts1,lag_ts2,lag_ts3

    def create_diff_ts_features(ts,diff_list):
        diff1_p1 = ts.diff(periods=diff_list[0])
        diff1_p2 = ts.diff(periods=diff_list[1])

        diff2_p1 = ts.diff(periods=diff_list[0]).diff()
        return diff1_p1,diff1_p2,diff2_p1

In [ ]:
dt = pd.Timestamp("2021-01-01") + pd.to_timedelta(range(df_inf.shape[0]), "H")
raw_feat_df = pd.DataFrame(columns=['time_idx','datetime','group','target','inflow_lag_0','inflow_lag_1','inflow_lag_2','inflow_lag_3','outflow_lag_0','outflow_lag_1','outflow_lag_2','outflow_lag_3'])

diff_list = [1,2]

In [ ]:
"""
stack all the raw feature time series of clusters vertically

tsfresh features are then added to these raw features to enlarge the feature set
"""

# inflow means supply, outflow means demand

for c in df_inf.columns:
    lags_list = zone_significant_lags[c]
    temp_df = pd.DataFrame(columns=['time_idx','datetime','group','target','inflow_lag_0','inflow_lag_1','inflow_lag_2','inflow_lag_3','outflow_lag_0','outflow_lag_1','outflow_lag_2','outflow_lag_3', 'inflow', "inflow_diff1_per1", 'inflow_diff1_per2', 'inflow_diff2_per1'])
    temp_df['time_idx'] = range(df_inf.shape[0])
    temp_df['datetime'] = dt
    temp_df['target'] = df_outf[c]
    temp_df['group'] = np.repeat(int(c),df_inf.shape[0])
    temp_df['inflow_lag_0'],temp_df['inflow_lag_1'],temp_df['inflow_lag_2'],temp_df['inflow_lag_3'] = create_raw_features.create_lag_features(df_inf[c],lags_list) # <-- do u need to change lags_list to supply ones -> no dont need
    temp_df['outflow_lag_0'],temp_df['outflow_lag_1'],temp_df['outflow_lag_2'],temp_df['outflow_lag_3'] = create_raw_features.create_lag_features(df_outf[c],lags_list)

    temp_df['inflow'] = df_inf[c]
    temp_df['inflow_diff1_per1'],temp_df['inflow_diff1_per2'], temp_df['inflow_diff2_per1'] = create_raw_features.create_diff_ts_features(df_inf[c],diff_list)

    raw_feat_df = pd.concat([raw_feat_df, temp_df],axis=0)

raw_feat_df = raw_feat_df.dropna().reset_index(drop=True)
raw_feat_df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,outflow_lag_2,outflow_lag_3,inflow,inflow_diff1_per1,inflow_diff1_per2,inflow_diff2_per1
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,0.0,0.0,4.0,-6.0,-6.0,-6.0
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,-3.0,-9.0,3.0
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,-3.0,3.0
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,0.0,1.0,3.0,2.0,2.0,2.0
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,-3.0,-1.0,-5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,10.0,93.0,153.0,-23.0,-17.0,-29.0
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,11.0,110.0,135.0,-18.0,-41.0,5.0
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,3.0,64.0,119.0,-16.0,-34.0,2.0
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,3.0,38.0,119.0,0.0,-16.0,16.0


In [ ]:
"""
Create TSFRESH Time series features

"""

win_lengths = [24] # hyperparameter

# drop all categorical columns

col_list = raw_feat_df.columns

feature_settings = {'fft_coefficient': [{'coeff': 0, 'attr': 'real'}, {'coeff': 1, 'attr': 'real'}],
                    #'mean': None,
                    #'standard_deviation': None,
                    'skewness': None,
                    'kurtosis': None,
                    #'minimum': None,
                    #'maximum': None,
                    #'sum_values': None,
                    #'variance': None,
                    #'absolute_sum_of_changes': None,
                    'count_above_mean': None,
                    'count_below_mean': None,
                    #'longest_strike_above_mean': None,
                    #'longest_strike_below_mean': None,
                    #'number_crossing_m': [{'m': 0}],
                    'autocorrelation': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
                    'partial_autocorrelation': [{'lag': 1}, {'lag': 2}, {'lag': 3}]}


for c in col_list[4:]: # select 1 numeric column at a time, leave out target and long lags
    print('time series name: ',c)
    for win in win_lengths:
        num_df = raw_feat_df.loc[:,['datetime','group',c]]
        print('num_df.shape', num_df.shape)
        df_rolled = tsf.utilities.dataframe_functions.roll_time_series(num_df,column_id='group',column_sort='datetime',max_timeshift=win)
        df_rolled.drop(columns=['group'], inplace=True)
        df_features = tsf.extract_features(df_rolled, column_id='id', column_sort='datetime',
                                            default_fc_parameters=feature_settings)
        del df_rolled # free-up the RAM

        # Remove any missing values
        imputed_features = impute(df_features)
        del df_features # free-up the RAM

        # Select a subset of relevant features using tsfresh's built-in feature selection function
        raw_feat_df.index= range(raw_feat_df.shape[0])
        imputed_features.index = range(raw_feat_df.shape[0])
        selected_features = tsf.select_features(imputed_features, raw_feat_df['target'])
        del imputed_features # free-up the RAM

        # Print the names of the selected features
        print('selected_features.columns',selected_features.columns)

        print('selected_features.columns: ',selected_features.columns)
        print('selected_features.shape: ',selected_features.shape)

        print('all_stns_df.columns: ',raw_feat_df.columns)
        print('all_stns_df.shape: ',raw_feat_df.shape)

        raw_feat_df = pd.concat([raw_feat_df,selected_features],axis=1)

        del num_df # free-up the RAM
        del selected_features # free-up the RAM

raw_feat_df.shape

time series name:  inflow_lag_0
num_df.shape (1631196, 3)


Feature Extraction: 100%|██████████| 20/20 [13:22<00:00, 40.10s/it]


selected_features.columns Index(['inflow_lag_0__fft_coefficient__attr_"real"__coeff_0',
       'inflow_lag_0__fft_coefficient__attr_"real"__coeff_1',
       'inflow_lag_0__skewness', 'inflow_lag_0__kurtosis',
       'inflow_lag_0__count_above_mean', 'inflow_lag_0__count_below_mean',
       'inflow_lag_0__autocorrelation__lag_1',
       'inflow_lag_0__autocorrelation__lag_2',
       'inflow_lag_0__autocorrelation__lag_3',
       'inflow_lag_0__partial_autocorrelation__lag_1',
       'inflow_lag_0__partial_autocorrelation__lag_2',
       'inflow_lag_0__partial_autocorrelation__lag_3'],
      dtype='object')
selected_features.columns:  Index(['inflow_lag_0__fft_coefficient__attr_"real"__coeff_0',
       'inflow_lag_0__fft_coefficient__attr_"real"__coeff_1',
       'inflow_lag_0__skewness', 'inflow_lag_0__kurtosis',
       'inflow_lag_0__count_above_mean', 'inflow_lag_0__count_below_mean',
       'inflow_lag_0__autocorrelation__lag_1',
       'inflow_lag_0__autocorrelation__lag_2',
       

Feature Extraction: 100%|██████████| 20/20 [13:28<00:00, 40.42s/it]


selected_features.columns Index(['inflow_lag_1__fft_coefficient__attr_"real"__coeff_0',
       'inflow_lag_1__fft_coefficient__attr_"real"__coeff_1',
       'inflow_lag_1__skewness', 'inflow_lag_1__kurtosis',
       'inflow_lag_1__count_above_mean', 'inflow_lag_1__count_below_mean',
       'inflow_lag_1__autocorrelation__lag_1',
       'inflow_lag_1__autocorrelation__lag_2',
       'inflow_lag_1__autocorrelation__lag_3',
       'inflow_lag_1__partial_autocorrelation__lag_1',
       'inflow_lag_1__partial_autocorrelation__lag_2',
       'inflow_lag_1__partial_autocorrelation__lag_3'],
      dtype='object')
selected_features.columns:  Index(['inflow_lag_1__fft_coefficient__attr_"real"__coeff_0',
       'inflow_lag_1__fft_coefficient__attr_"real"__coeff_1',
       'inflow_lag_1__skewness', 'inflow_lag_1__kurtosis',
       'inflow_lag_1__count_above_mean', 'inflow_lag_1__count_below_mean',
       'inflow_lag_1__autocorrelation__lag_1',
       'inflow_lag_1__autocorrelation__lag_2',
       

...

Feature Extraction: 100%|██████████| 20/20 [13:37<00:00, 40.87s/it]


selected_features.columns Index(['inflow_diff1_per2__fft_coefficient__attr_"real"__coeff_0',
       'inflow_diff1_per2__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff1_per2__skewness', 'inflow_diff1_per2__kurtosis',
       'inflow_diff1_per2__count_above_mean',
       'inflow_diff1_per2__count_below_mean',
       'inflow_diff1_per2__autocorrelation__lag_1',
       'inflow_diff1_per2__autocorrelation__lag_2',
       'inflow_diff1_per2__autocorrelation__lag_3',
       'inflow_diff1_per2__partial_autocorrelation__lag_1',
       'inflow_diff1_per2__partial_autocorrelation__lag_2',
       'inflow_diff1_per2__partial_autocorrelation__lag_3'],
      dtype='object')
selected_features.columns:  Index(['inflow_diff1_per2__fft_coefficient__attr_"real"__coeff_0',
       'inflow_diff1_per2__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff1_per2__skewness', 'inflow_diff1_per2__kurtosis',
       'inflow_diff1_per2__count_above_mean',
       'inflow_diff1_per2__count_below_mean

Feature Extraction: 100%|██████████| 20/20 [13:24<00:00, 40.24s/it]


selected_features.columns Index(['inflow_diff2_per1__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff2_per1__count_above_mean',
       'inflow_diff2_per1__autocorrelation__lag_1',
       'inflow_diff2_per1__autocorrelation__lag_2',
       'inflow_diff2_per1__partial_autocorrelation__lag_1',
       'inflow_diff2_per1__partial_autocorrelation__lag_2',
       'inflow_diff2_per1__partial_autocorrelation__lag_3',
       'inflow_diff2_per1__fft_coefficient__attr_"real"__coeff_0',
       'inflow_diff2_per1__autocorrelation__lag_3',
       'inflow_diff2_per1__count_below_mean', 'inflow_diff2_per1__kurtosis'],
      dtype='object')
selected_features.columns:  Index(['inflow_diff2_per1__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff2_per1__count_above_mean',
       'inflow_diff2_per1__autocorrelation__lag_1',
       'inflow_diff2_per1__autocorrelation__lag_2',
       'inflow_diff2_per1__partial_autocorrelation__lag_1',
       'inflow_diff2_per1__partial_autocorrelation__l

(1631196, 159)

In [ ]:
for c in raw_feat_df.columns:
    print(c)

time_idx
datetime
group
target
inflow_lag_0
inflow_lag_1
inflow_lag_2
inflow_lag_3
outflow_lag_0
outflow_lag_1
outflow_lag_2
outflow_lag_3
inflow
inflow_diff1_per1
inflow_diff1_per2
inflow_diff2_per1
inflow_lag_0__fft_coefficient__attr_"real"__coeff_0
inflow_lag_0__fft_coefficient__attr_"real"__coeff_1
inflow_lag_0__skewness
inflow_lag_0__kurtosis
inflow_lag_0__count_above_mean
inflow_lag_0__count_below_mean
inflow_lag_0__autocorrelation__lag_1
inflow_lag_0__autocorrelation__lag_2
inflow_lag_0__autocorrelation__lag_3
inflow_lag_0__partial_autocorrelation__lag_1
inflow_lag_0__partial_autocorrelation__lag_2
inflow_lag_0__partial_autocorrelation__lag_3
inflow_lag_1__fft_coefficient__attr_"real"__coeff_0
inflow_lag_1__fft_coefficient__attr_"real"__coeff_1
inflow_lag_1__skewness
inflow_lag_1__kurtosis
inflow_lag_1__count_above_mean
inflow_lag_1__count_below_mean
inflow_lag_1__autocorrelation__lag_1
inflow_lag_1__autocorrelation__lag_2
inflow_lag_1__autocorrelation__lag_3
inflow_lag_1__parti

In [ ]:
raw_feat_df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,inflow_diff2_per1__count_above_mean,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_3,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0",inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__kurtosis
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,0.0,-0.517121,0.028399,-0.517109,-0.362827,-0.207694,-6.0,-0.000421,0.0,0.574259
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,1.0,-1.000000,0.028399,-0.517109,-0.362827,-0.207694,-3.0,-0.000421,1.0,0.574259
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,2.0,-0.250000,-1.000000,-0.517109,-0.362827,-0.207694,0.0,-0.000421,1.0,0.574259
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,3.0,-0.146199,-0.438596,-0.146199,-0.362827,-0.207694,2.0,-0.684211,1.0,3.750693
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,3.0,-0.131773,-0.532020,-0.131773,-0.362827,-0.207694,-3.0,-0.919951,2.0,-3.126259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,14.0,-0.421845,-0.103924,-0.421845,-0.342896,0.203789,-64.0,0.351349,11.0,0.235154
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,13.0,-0.546637,0.012524,-0.546637,-0.408289,-0.018269,10.0,0.202870,12.0,0.405906
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,14.0,-0.540946,0.001364,-0.540946,-0.411744,-0.030810,14.0,0.203590,11.0,0.406788
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,14.0,-0.443736,-0.035146,-0.443736,-0.288941,0.078741,-8.0,0.206272,11.0,0.743705


In [ ]:
pip install pyarrow

In [ ]:
raw_feat_df.to_parquet('combined_dd_ss_raw_feat_data_significant_lags_pruned.parquet')

In [ ]:
import pandas as pd
import os
os.chdir('/content/drive/MyDrive/FYP_Python_Notebooks')
df = pd.read_parquet('[Draft_2]_combined_dd_ss_raw_feat_data_significant_lags_pruned.parquet', engine='pyarrow')
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,inflow_diff2_per1__count_above_mean,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_3,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0",inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__kurtosis
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,0.0,-0.517121,0.028399,-0.517109,-0.362827,-0.207694,-6.0,-0.000421,0.0,0.574259
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,1.0,-1.000000,0.028399,-0.517109,-0.362827,-0.207694,-3.0,-0.000421,1.0,0.574259
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,2.0,-0.250000,-1.000000,-0.517109,-0.362827,-0.207694,0.0,-0.000421,1.0,0.574259
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,3.0,-0.146199,-0.438596,-0.146199,-0.362827,-0.207694,2.0,-0.684211,1.0,3.750693
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,3.0,-0.131773,-0.532020,-0.131773,-0.362827,-0.207694,-3.0,-0.919951,2.0,-3.126259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,14.0,-0.421845,-0.103924,-0.421845,-0.342896,0.203789,-64.0,0.351349,11.0,0.235154
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,13.0,-0.546637,0.012524,-0.546637,-0.408289,-0.018269,10.0,0.202870,12.0,0.405906
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,14.0,-0.540946,0.001364,-0.540946,-0.411744,-0.030810,14.0,0.203590,11.0,0.406788
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,14.0,-0.443736,-0.035146,-0.443736,-0.288941,0.078741,-8.0,0.206272,11.0,0.743705


In [ ]:
df['_hour_of_day'] = df["datetime"].dt.hour.astype(str)
df['_day_of_week'] = df["datetime"].dt.dayofweek.astype(str)
df['_day_of_month'] = df["datetime"].dt.day.astype(str)
df['_day_of_year'] = df["datetime"].dt.dayofyear.astype(str)
df['_week_of_year'] = df["datetime"].dt.weekofyear.astype(str)
df['_month_of_year'] = df["datetime"].dt.month.astype(str)
df['_year'] = df["datetime"].dt.year.astype(str)

<ipython-input-4-b9431483d75c>:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['_week_of_year'] = df["datetime"].dt.weekofyear.astype(str)


In [ ]:
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__kurtosis,_hour_of_day,_day_of_week,_day_of_month,_day_of_year,_week_of_year,_month_of_year,_year
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,-0.000421,0.0,0.574259,23,3,7,7,1,1,2021
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,-0.000421,1.0,0.574259,0,4,8,8,1,1,2021
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,-0.000421,1.0,0.574259,1,4,8,8,1,1,2021
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,-0.684211,1.0,3.750693,2,4,8,8,1,1,2021
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,-0.919951,2.0,-3.126259,3,4,8,8,1,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,0.351349,11.0,0.235154,16,1,31,304,44,10,2023
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,0.202870,12.0,0.405906,17,1,31,304,44,10,2023
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,0.203590,11.0,0.406788,18,1,31,304,44,10,2023
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,0.206272,11.0,0.743705,19,1,31,304,44,10,2023


In [ ]:
total_time_idx = df[df.group == 4].shape[0]
print(total_time_idx)

24646


In [ ]:
total_time_idx = df[df.group == 263].shape[0]
print(total_time_idx)

24646


In [ ]:
print(df['time_idx'][0])

167
